Import files

In [13]:
import collections
import json
import re
import string
import nltk
import http.client
import aiohttp
import asyncio, xmlrpc.client
path = "why.txt"
out_path = "freq_table.json"
definitions_path = "defs.json"
sentences_path = "sentences.txt"
datayze_url = "https://datayze.com"
extra_defs = "extra_defs.txt"

wordCount = 10
thresholdFreq = 5
word_definitions = {}
tasks = []
wordsToLookup = set()
wordsLeftToLookup = set()
extra_words = []
server = xmlrpc.client.ServerProxy(
    "http://localhost:{}".format(8000))

Preprocess

In [14]:

f = open(path, "r")
content = f.read()
sentences = nltk.sent_tokenize(content.replace("\n", " "))

res = [x.lower().encode("ascii", "ignore").decode("ascii") for x in re.sub('['+string.punctuation+']', '', content).split()]
freq = dict(collections.Counter(res))


Word lookup

In [15]:

def getWord(k, v):
    global wordCount
    if v > thresholdFreq: return
    word_rank = server.predict(k)
    print(f"Looking {k}: #{word_rank}")
    if word_rank < 0.4: return
    wordsToLookup.add(k)
async def getWordDefinition(session : aiohttp.ClientSession, k):
    async with session.get(f"https://api.dictionaryapi.dev/api/v2/entries/en/{k}") as response:
        print("Status: {} and reason: {}".format(response.status, response.reason))
        res = await response.json()
        if (response.status == 404): return
        res_def = res[0]
        word_definitions[k] = res_def
        wordsLeftToLookup.remove(k)

def generateLookupWords():
    for k, v in freq.items():
        getWord(k, v)

def resetLookup():
    global wordsLeftToLookup
    wordsLeftToLookup = set(wordsToLookup)
async def getFromWeb():
    try:
        connector = aiohttp.TCPConnector(limit=3, limit_per_host=2)
        mysession = aiohttp.ClientSession(connector=connector)
        async with mysession as session:
            for k in wordsLeftToLookup:
                tasks.append(asyncio.ensure_future(getWordDefinition(session, k)))
            res = await asyncio.gather(*tasks)
    except Exception as e:
        print(e)
    # Save it
    dictf = open(definitions_path, "w")
    dictf.write(json.dumps(word_definitions))
    dictf.close()

In [16]:
generateLookupWords()

Looking praise: #0.029892692927184404
Looking contribution: #0.029892692927184404
Looking debate: #0.008615845672597844
Looking similarlooking: #0.23355263168749998
Looking multiplicity: #0.16447368437499998
Looking accidental: #0.04552793030061346
Looking openness: #0.04552793030061346
Looking willingness: #0.029892692927184404
Looking kenneth: #0.44736842110294095
Looking arrow: #0.11368421063999996
Looking authors: #0.029892692927184404
Looking convincingly: #0.04552793030061346
Looking escape: #0.029892692927184404
Looking appropriate: #0.008615845672597844
Looking intimate: #0.029892692927184404
Looking vitality: #0.16447368437499998
Looking questions: #0.008615845672597844
Looking gary: #0.16666666661111107
Looking becker: #0.44736842110294095
Looking insightful: #0.04552793030061346
Looking packed: #0.029892692927184404
Looking reviews: #0.029892692927184404
Looking spiral: #0.04552793030061346
Looking missed: #0.029892692927184404
Looking manmade: #0.44736842110294095
Looking f

In [17]:
def addWordsRoutine():
    global extra_words
    extra_words = open(extra_defs, "r").read().split(",")
    map(wordsLeftToLookup.add, extra_words)
    word=input("Add a word (q to quit): ")
    while word != "q":
        wordsToLookup.add(word)
        wordsLeftToLookup.add(word)
        extra_words.append(word)
        word=input("Add a word (q to quit): ")
    with open(extra_defs, "w") as w:
        w.write(",".join(extra_words))
addWordsRoutine()

In [18]:
resetLookup()

In [19]:
await getFromWeb()

Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 200 and reason: OK
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason: Not Found
Status: 404 and reason

Load cached file

In [20]:
dictf = open(definitions_path, "r")
word_definitions = json.loads(dictf.read())
dictf.close()

In [21]:
def trunc(s: str, target: str, l: int):
    if len(s) <= l: return s
    i = s.find(target)
    if i == -1: return s
    left = s.rfind(' ', 0, i - l) if i - l >= 0 else 0
    right = s.find(' ', i + l) if i + l <= len(s) else len(s)
    # print(i, left, right)
    if left == -1: left = 0
    if right == -1: right = len(s)
    return f"{'... ' if left != 0 else ''}{s[left:right]}{' ...' if right != len(s) else ''}"
print(trunc("Hello world! This is a target word test.", "is", 10))
def example_sentence(s: str, k: str):
    return trunc(s, k, 50).replace(k, f'*{k}*')
def get_page(k: str):
    containedSentence = [(i, s) for i, s in enumerate(sentences) if k in s.lower()]
    if len(containedSentence) > 0:
        return int(containedSentence[0][0] / 10) + 1
    return -1

Hello world! This is a target ...


Generate

In [22]:
sortedDefs = sorted(word_definitions.items(), key=lambda x: get_page(x[0]))
for k, v in sortedDefs:
    containedSentence = [s for s in sentences if k in s.lower()]
    phonetic = f" *{v['phonetic']}*" if "phonetic" in v.keys() else ""
    page_num = get_page(k)
    page = f" Page {page_num}" if page_num != -1 else ""
    print(f"**{k}**{phonetic}{page}: ")
    if len(containedSentence) > 0:
        print(f'- "{example_sentence(containedSentence[0], k)}"')
    for meaning in v['meanings']:
        maxMeanings = 3
        form = meaning["partOfSpeech"]
        print(f"- {form}:")
        for definition in meaning["definitions"]:
            print(f"  - {definition['definition']}")
            if "example" in definition.keys():
                print(f"    E.g. \"{example_sentence(definition['example'], k)}\"")
            maxMeanings -= 1
            if maxMeanings <= 0: break
    print()


**maladies**: 
- noun:
  - Any ailment or disease of the body; especially, a lingering or deep-seated disorder.
  - A moral or mental defect or disorder.

**runup**: 
- noun:
  - The approach run of a bowler before delivering the ball
  - The approach run of a high jumper or other athlete in order to gather speed or momentum
  - A period of time just before an important event.
    E.g. "The candidates were very nervous in the run-up to the election."

**webers**: 
- noun:
  - In the International System of Units, the derived unit of magnetic flux; the flux linking a circuit of one turn that produces an electromotive force of one volt when reduced uniformly to zero in one second. Symbol: Wb.

**anachronism**: 
- noun:
  - A chronological mistake; the erroneous dating of an event, circumstance, or object.
  - A person or thing which seems to belong to a different time or period of time.

**noone**: 
- pronoun:
  - Used in contrast to anyone, someone or everyone: not one person; nobody.
 